In [1]:
# !git clone https://github.com/Trustworthy-ML-Lab/CB-LLMs.git

Cloning into 'CB-LLMs'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 100 (delta 41), reused 56 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (100/100), 921.58 KiB | 5.18 MiB/s, done.
Resolving deltas: 100% (41/41), done.
/home/mac043/teams/dsmlp/Text Generation/CB-LLMs


## Generation

### 1. Setup

In [1]:
%%time

%cd generation
%pip install -r requirements.txt
%pip install torchvision==0.19.0

/home/mac043/teams/dsmlp/Concept-Bottleneck-LLM/generation
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
CPU times: user 61.5 ms, sys: 30.9 ms, total: 92.4 ms
Wall time: 6.74 s


In [ ]:
%%time

# Download finetuned CB-LLMs
!git lfs install
!git clone https://huggingface.co/cesun/cbllm-generation temp_repo
!mv temp_repo/from_pretained_llama3_lora_cbm .
!rm -rf temp_repo

Updated git hooks.
Git LFS initialized.
Cloning into 'temp_repo'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 37 (delta 8), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (37/37), 8.02 KiB | 820.00 KiB/s, done.
Filtering content: 100% (8/8), 1.68 GiB | 38.05 MiB/s, done.
CPU times: user 255 ms, sys: 44.4 ms, total: 299 ms
Wall time: 47.4 s


In [7]:
from huggingface_hub import notebook_login
notebook_login()

# !huggingface-cli login
!huggingface-cli whoami

MarcusChang


### 2. Training
Models:
- `meta-llama/Llama-3.2-3B-Instruct`: https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct

In [3]:
import torch

# Clear CUDA memory to avoid OOM errors
torch.cuda.reset_peak_memory_stats()

In [5]:
%%time

# Train a CB-LLM
!python train_CBLLM.py \
    --dataset 'SetFit/sst2' \
    --model_id 'meta-llama/Llama-3.2-3B-Instruct'

/home/mac043/.local/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
loading data...
Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.
training data len:  6920
val data len:  872
tokenizing...
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/huggingface_hub/utils/_http.py", line 406, in hf_raise_for_status
    response.raise_for_status()
  File "/opt/conda/lib/python3.11/site-packages/requests/models.py", line 1024, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct/resolve/main/config.json

The above exception was the direct cause of the following exception:

Traceback (most re

In [ ]:
# Record peak memory usage
peak_memory = torch.cuda.max_memory_allocated() / 1024**2  # MB
print(f"Peak allocated memory: {peak_memory:.2f} MB")

peak_reserved = torch.cuda.max_memory_reserved() / 1024**2  # MB
print(f"Peak reserved memory: {peak_reserved:.2f} MB")

### 3. Testing

In [ ]:
%%time

# Test the concept detection
!python test_concepts.py

2025-05-13 22:34:58.909933: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747175698.929656    7966 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747175698.935695    7966 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-13 22:34:58.956044: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
loading data...
Repo card metadata block was not found. Setting CardData to empty.
test data len:  1821
tokenizing...
config.

In [ ]:
%%time

# Test the steerability
!python train_classifier.py
!python test_steerability.py

In [ ]:
%%time

# Test the perplexity
!python test_perplexity.py

In [ ]:
%%time

# Visualize the top 10 tokens with the highest weight connect to a concept neuron
!python test_weight.py

In [ ]:
%%time

# Intervene the concept neurons and generate a sentence
!python test_generation.py